In [1]:
import pandas as pd
import CococoNet_reader 
import itertools
import numpy as np
import Name_resolver
import warnings

In [30]:
def Calculate_Score_list_for_thresholding(Species_1 = 'rice'): ### THIS IS TOMATO SPECIFIC
    

    import pandas as pd
    import CococoNet_reader
    import Name_resolver

    Species_2 = 'tomato'
    #Get Species Names in Common form 
    common_name_1 = Name_resolver.species_name_resolver(Species_1,'common')
    common_name_2 = 'tomato'


    matrix_of_locations = pd.read_csv('/data/passala/git/Coexpressalog_Method_Development/Cross_all_species/map_of_tomato_locations.csv', index_col= 0)#List of cross species maps
    current_location = matrix_of_locations["tomato"][common_name_1] #get string for map

    cross_species_n_m_genes = pd.read_csv('/data/passala/git/Solanum_SV_Pan_Genome/Jack_prickles/rice_tomato_added_log.csv')

    if current_location.index('tomato') == 30:
        cross_species_n_m_genes = cross_species_n_m_genes.rename(columns = {'sp1_gene':'tomato','sp2_gene':common_name_1})
    else:
        cross_species_n_m_genes = cross_species_n_m_genes.rename(columns = {'sp1_gene':common_name_1,'sp2_gene':"tomato"})
        

    ### Get one to ones
    cross_species_map_one_to_one = cross_species_n_m_genes.drop_duplicates(subset=common_name_1, keep= False,)
    cross_species_map_one_to_one = cross_species_map_one_to_one.drop_duplicates(subset= common_name_2, keep= False)

    ## Convert to Dictionary
    dictionary_mapper_one_to_two = cross_species_map_one_to_one.set_index(common_name_1).to_dict()[common_name_2]
    dictionary_mapper_dos_to_uno = cross_species_map_one_to_one.set_index(common_name_2).to_dict()[common_name_1]

    ## Read In Cococonets 
    coconet_species_one = CococoNet_reader.read_cococonet(common_name_1)
    coconet_species_two = CococoNet_reader.read_cococonet(common_name_2)

    cross_species_n_m_genes['Group ID'] = 'Unassigned'
    

    ## Assign Genes to Groups
    id_indexer = 0
    for gene_pair in cross_species_n_m_genes.iterrows():
        
        if gene_pair[1]['Group ID'] == 'Unassigned':
            current_species_1_gene = gene_pair[1][common_name_1]
            current_species_2_gene = gene_pair[1][common_name_2]
            cross_species_n_m_genes['Group ID'].loc[(cross_species_n_m_genes[common_name_1] == current_species_1_gene) & (cross_species_n_m_genes['Group ID'] == 'Unassigned')] = id_indexer
            cross_species_n_m_genes['Group ID'].loc[(cross_species_n_m_genes[common_name_2] == current_species_2_gene) & (cross_species_n_m_genes['Group ID'] == 'Unassigned')] = id_indexer

            all_labeled_groups = cross_species_n_m_genes.loc[cross_species_n_m_genes['Group ID'] == id_indexer]

            all_labeled_groups_species_1_genes = all_labeled_groups[common_name_1].to_list()
            all_labeled_groups_species_2_genes = all_labeled_groups[common_name_2].to_list()

            cross_species_n_m_genes['Group ID'].loc[cross_species_n_m_genes[common_name_1].isin(all_labeled_groups_species_1_genes)] = id_indexer
            cross_species_n_m_genes['Group ID'].loc[cross_species_n_m_genes[common_name_2].isin(all_labeled_groups_species_2_genes)] = id_indexer

            id_indexer += 1



    #Identify Pairs for evaluation
    all_pairs_to_evaluate_for_functional_conservation = pd.DataFrame(columns = [common_name_1,common_name_2,'Group Number'])
    for group_number in list(set(cross_species_n_m_genes['Group ID'].to_list())):
        current_gene_map = cross_species_n_m_genes.loc[cross_species_n_m_genes['Group ID'] == group_number]
        list_of_species_1_genes_in_group = list(set(current_gene_map[common_name_1].to_list()))
        list_of_species_2_genes_in_group = list(set(current_gene_map[common_name_2].to_list()))
        all_combo_list_current_genes = itertools.product(list_of_species_1_genes_in_group,list_of_species_2_genes_in_group)
        all_combo_list_current_genes = list(map(list,all_combo_list_current_genes))
        current_list_of_pairs = pd.DataFrame(all_combo_list_current_genes,columns = [Species_1,Species_2])
        current_list_of_pairs['Group Number'] = group_number
        all_pairs_to_evaluate_for_functional_conservation = all_pairs_to_evaluate_for_functional_conservation.append(current_list_of_pairs)


    
    all_pairs_to_evaluate_for_functional_conservation['Species 1 Score'] = np.nan
    all_pairs_to_evaluate_for_functional_conservation['Species 2 Score'] = np.nan


    ## Trim cococonets to match


    trimmed_species_1_cococonet = coconet_species_one[coconet_species_one.columns.intersection(cross_species_n_m_genes[common_name_1].to_list())]
    trimmed_species_1_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.index.isin(cross_species_n_m_genes[common_name_1].to_list())]
    double_species_1_trimmed_cococonet = trimmed_species_1_cococonet[trimmed_species_1_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_1].to_list())]
    double_species_1_trimmed_cococonet = double_species_1_trimmed_cococonet.replace(1,0)

    trimmed_species_2_cococonet = coconet_species_two[coconet_species_two.columns.intersection(cross_species_n_m_genes[common_name_2].to_list())]
    trimmed_species_2_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.index.isin(cross_species_n_m_genes[common_name_2].to_list())]
    double_species_2_trimmed_cococonet = trimmed_species_2_cococonet[trimmed_species_2_cococonet.columns.intersection(cross_species_map_one_to_one[common_name_2].to_list())]
    double_species_2_trimmed_cococonet = double_species_2_trimmed_cococonet.replace(1,0)


    ## Rank
    species_1_cococonet_ranked = trimmed_species_1_cococonet.rank()
    species_2_cococonet_ranked = trimmed_species_2_cococonet.rank()

    #Do top 10 Genes
    top_10_species_1_genes = np.array(
        [double_species_1_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_1_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_1_genes_dataframe = pd.DataFrame(
        data=top_10_species_1_genes,
        index=double_species_1_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )

    #Convert 
    top_10_species_1_genes_as_species_2 = top_10_species_1_genes_dataframe.replace(to_replace=dictionary_mapper_one_to_two)

    # Get genes for checking 
    have_species_1_pairs = all_pairs_to_evaluate_for_functional_conservation.loc[all_pairs_to_evaluate_for_functional_conservation[common_name_1].isin(top_10_species_1_genes_as_species_2.index)]
    trimmed_all_gene_pairs_for_fc = have_species_1_pairs.loc[have_species_1_pairs[common_name_2].isin(trimmed_species_2_cococonet.index)]
    trimmed_all_gene_pairs_for_fc = trimmed_all_gene_pairs_for_fc.reset_index(drop = True)

    # Get values in species 2 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][common_name_1]
        current_species_2_gene = two_genes[1][common_name_2]
        finger_print_genes = top_10_species_1_genes_as_species_2.loc[current_species_1_gene].to_list()
        gene_ranks_in_species_2 = species_2_cococonet_ranked.loc[species_2_cococonet_ranked.index.isin(finger_print_genes), current_species_2_gene]
        avg_rank_in_species_2 = gene_ranks_in_species_2.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.at[index_from_pairs, 'Species 1 Score'] = avg_rank_in_species_2

    #Repeat for Species 2 

    top_10_species_2_genes = np.array(
        [double_species_2_trimmed_cococonet.T[c].nlargest(10).index.values for c in double_species_2_trimmed_cococonet.T]
    )  # using pair list above, cut down top 10 list to relevant genes, probably by adding list as a column in panda and then filtering panda to index of pair list
    top_10_species_2_genes_dataframe = pd.DataFrame(
        data=top_10_species_2_genes,
        index=double_species_2_trimmed_cococonet.index,
        columns=[
            "One",
            "Two",
            "Three",
            "Four",
            "Five",
            "Six",
            "Seven",
            "Eight",
            "Nine",
            "Ten",
        ],
    )
    

    #convert 
    top_10_species_2_genes_as_species_1 = top_10_species_2_genes_dataframe.replace(to_replace=dictionary_mapper_dos_to_uno)


    # Get values in species 1 
    for two_genes in trimmed_all_gene_pairs_for_fc.iterrows():
        current_species_1_gene = two_genes[1][common_name_1]
        current_species_2_gene = two_genes[1][common_name_2]
        finger_print_genes = top_10_species_2_genes_as_species_1.loc[current_species_2_gene].to_list()
        gene_ranks_in_species_1 = species_1_cococonet_ranked.loc[species_1_cococonet_ranked.index.isin(finger_print_genes), current_species_1_gene]
        avg_rank_in_species_1 = gene_ranks_in_species_1.mean()
        index_from_pairs = two_genes[0]
        trimmed_all_gene_pairs_for_fc.loc[index_from_pairs, 'Species 2 Score'] = avg_rank_in_species_1
    
    #Caluclate Divisors 
    Number_of_species_1_genes = len(top_10_species_1_genes_as_species_2)
    Number_of_species_2_genes = len(top_10_species_2_genes_as_species_1)


    species_1_score_divisor = Number_of_species_2_genes - 4.5
    species_2_score_divisor = Number_of_species_1_genes-4.5

    #Divide and Average 
    trimmed_all_gene_pairs_for_fc['Species 1 Score'] = trimmed_all_gene_pairs_for_fc['Species 1 Score']/species_1_score_divisor
    trimmed_all_gene_pairs_for_fc['Species 2 Score'] = trimmed_all_gene_pairs_for_fc['Species 2 Score']/species_2_score_divisor
    trimmed_all_gene_pairs_for_fc['Total Score'] = trimmed_all_gene_pairs_for_fc[['Species 1 Score','Species 2 Score']].mean(axis = 1)
    
    return trimmed_all_gene_pairs_for_fc
    

In [4]:
def Threshold_and_generate_coexpressalog_list(trimmed_all_gene_pairs_for_fc, single_pair_junk_threshold = .8, many_to_many_junk_threshold = .85,difference_between_many_to_drop = .03):
    
    
    list_of_true_pairs = []
    list_of_genes_to_average_and_set_to_be_equal = []


    common_name_1 = trimmed_all_gene_pairs_for_fc.columns[0]
    common_name_2 = trimmed_all_gene_pairs_for_fc.columns[1]

    for current_group in list(set(trimmed_all_gene_pairs_for_fc['Group Number'].to_list())):
        dataframe_of_group = trimmed_all_gene_pairs_for_fc.loc[trimmed_all_gene_pairs_for_fc['Group Number'] == current_group]
        dataframe_of_group = dataframe_of_group[[common_name_1,common_name_2,'Total Score']]
        wide_format = dataframe_of_group.pivot(index = common_name_1,columns= common_name_2,values= 'Total Score')
        if (wide_format<single_pair_junk_threshold).all(axis = None):
            continue
        elif wide_format.shape ==(0,0):
            continue
        elif wide_format.shape == (1,1): # Group is 1 to 1, retain Format: Arabidopsis then Maize
            one_true_pair = [wide_format.index.item(),wide_format.columns.item()]
            list_of_true_pairs.append(one_true_pair)
        elif len(wide_format)>1 and len(wide_format.columns) ==1:
            one_true_pair = [wide_format.idxmax(axis =0).item(),wide_format.columns.item()]
            list_of_true_pairs.append(one_true_pair)
        elif len(wide_format) == 1 and len(wide_format.columns)>1:
            one_true_pair = [wide_format.index.item(), wide_format.idxmax(axis = 1).item()]
            list_of_true_pairs.append(one_true_pair)
        else:

            #Drop Low Quality Columns and Rows
            cols_to_drop = wide_format.columns[wide_format.max()<many_to_many_junk_threshold]
            wide_format = wide_format.drop(columns= cols_to_drop)
            rows_to_drop = wide_format.index[wide_format.max(axis = 1)<many_to_many_junk_threshold]
            wide_format = wide_format.drop(index= rows_to_drop)
            if (wide_format<single_pair_junk_threshold).all(axis = None):
                continue
            elif wide_format.shape ==(0,0):
                continue
            elif wide_format.shape == (1,1): # Group is 1 to 1, retain Format: Arabidopsis then Maize
                one_true_pair = [wide_format.index.item(),wide_format.columns.item()]
                list_of_true_pairs.append(one_true_pair)
            elif len(wide_format)>1 and len(wide_format.columns) ==1:
                one_true_pair = [wide_format.idxmax(axis =0).item(),wide_format.columns.item()]
                list_of_true_pairs.append(one_true_pair)
            elif len(wide_format) == 1 and len(wide_format.columns)>1:
                one_true_pair = [wide_format.index.item(), wide_format.idxmax(axis = 1).item()]
                list_of_true_pairs.append(one_true_pair)
            elif (wide_format>.9).all(axis = None):
                ### Put in retention code here
                both_gene_lists_to_average = [wide_format.index.to_list(), wide_format.columns.to_list()]
                list_of_genes_to_average_and_set_to_be_equal.append(both_gene_lists_to_average)
                
            else:
                for cur_row in wide_format.iterrows():
                    cur_row_max = cur_row[1].max()
                    cur_row[1][cur_row[1]< cur_row_max - difference_between_many_to_drop] = np.nan
                    wide_format.loc[cur_row[0]] = cur_row[1]
                wide_format = wide_format.dropna(axis = 1, how = 'all')
                for cur_col in wide_format.columns:
                    cur_col_max = wide_format[cur_col].max()
                    wide_format[cur_col].loc[wide_format[cur_col]< cur_col_max-difference_between_many_to_drop] = np.nan
                wide_format = wide_format.dropna(axis = 0, how = 'all')
                col_count = wide_format.count() == 1
                wide_format = wide_format.loc[:,col_count]
                row_count = wide_format.count(axis = 1) ==1 
                wide_format = wide_format.loc[row_count,:]
                wide_format = wide_format.dropna(axis = 1, how = 'all')
                wide_format = wide_format.dropna(axis = 0, how = 'all')
                for label,content in wide_format.items():
                    cur_species_2_label = label
                    cur_species_1_label = content.idxmax()
                    if type(cur_species_1_label) == str:
                        one_true_pair = [cur_species_1_label,cur_species_2_label]
                        list_of_true_pairs.append(one_true_pair)


    true_pair_dataframe = pd.DataFrame(columns= [f'{common_name_1} gene',f'{common_name_2} gene'], data = list_of_true_pairs)
    true_pair_dataframe = true_pair_dataframe.drop_duplicates(subset = f'{common_name_1} gene')
    true_pair_dataframe = true_pair_dataframe.drop_duplicates(subset = f'{common_name_2} gene')
    
    return true_pair_dataframe
                

In [5]:
# tomato_cococonet_full = CococoNet_reader.read_cococonet('tomato')
# tomato_cococonet_full

In [31]:
score_list = Calculate_Score_list_for_thresholding('rice')

/tmp/ipykernel_515012/529480882.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cross_species_n_m_genes['Group ID'].loc[(cross_species_n_m_genes[common_name_1] == current_species_1_gene) & (cross_species_n_m_genes['Group ID'] == 'Unassigned')] = id_indexer
/tmp/ipykernel_515012/529480882.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cross_species_n_m_genes['Group ID'].loc[(cross_species_n_m_genes[common_name_2] == current_species_2_gene) & (cross_species_n_m_genes['Group ID'] == 'Unassigned')] = id_indexer
/tmp/ipykernel_515012/529480882.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [32]:
score_list

,rice,tomato,Group Number,Species 1 Score,Species 2 Score,Total Score
0,Os03g0731500,LOC101248771,0,0.863765,0.711618,0.787692
1,Os04g0195000,LOC101265892,1,0.826340,0.837133,0.831736
2,Os03g0290900,LOC101266827,2,0.787203,0.917940,0.852572
3,Os03g0290900,LOC101256391,2,0.680071,0.786374,0.733223
4,Os04g0643300,LOC101249057,4,0.905174,0.739110,0.822142
...,...,...,...,...,...,...
18779,Os11g0636900,LOC101263485,18684,0.936167,0.896656,0.916411
18780,Os11g0636900,LOC101251944,18684,0.925166,0.870218,0.897692
18781,Os01g0238900,LOC101251902,18687,0.962566,0.968313,0.965440
18782,Os10g0197000,LOC101256309,18688,0.724539,0.623705,0.674122


In [33]:
score_list.loc[score_list['tomato']=='LOG8']

,rice,tomato,Group Number,Species 1 Score,Species 2 Score,Total Score
5971,Os02g0628000,LOG8,5967,0.563431,0.271626,0.417528
5979,Os03g0857900,LOG8,5967,0.370414,0.329897,0.350156
5987,Os03g0109300,LOG8,5967,0.545955,0.486035,0.515995
5995,Os10g0479500,LOG8,5967,0.558583,0.460796,0.509690
6003,Os05g0541200,LOG8,5967,0.613619,0.674078,0.643849
6011,Os03g0697200,LOG8,5967,0.598754,0.548886,0.573820
6019,Os01g0588900,LOG8,5967,0.845909,0.825260,0.835584
6027,Os09g0547500,LOG8,5967,0.345642,0.146299,0.245970
6035,Os04g0518800,LOG8,5967,0.400000,0.215995,0.307997
6043,Os05g0591600,LOG8,5967,0.422374,0.181658,0.302016


In [34]:
log_family = score_list.loc[score_list['Group Number'] == 5967]

In [35]:
log_family.head()

,rice,tomato,Group Number,Species 1 Score,Species 2 Score,Total Score
5967,Os02g0628000,LOC101262778,5967,0.588237,0.283072,0.435654
5968,Os02g0628000,LOC101252798,5967,0.447731,0.573833,0.510782
5969,Os02g0628000,LOG6,5967,0.677486,0.393145,0.535316
5970,Os02g0628000,LOG7,5967,0.397330,0.487683,0.442507
5971,Os02g0628000,LOG8,5967,0.563431,0.271626,0.417528


In [37]:
log_family.pivot(columns = 'rice', index = 'tomato', values = 'Total Score')

rice,Os01g0588900,Os01g0708500,Os02g0628000,Os03g0109300,Os03g0697200,Os03g0857900,Os04g0518800,Os05g0541200,Os05g0591600,Os09g0547500,Os10g0479500
tomato,,,,,,,,,,,
LOC101244985,0.504033,0.740777,0.434459,0.604588,0.347234,0.640599,0.406724,0.495716,0.646969,0.773244,0.319957
LOC101246083,0.529154,0.362951,0.554862,0.410157,0.676349,0.346513,0.645364,0.438722,0.282994,0.303900,0.719797
LOC101252798,0.406399,0.787809,0.510782,0.657427,0.435707,0.450192,0.589088,0.592613,0.643840,0.465222,0.493828
LOC101262778,0.826898,0.355101,0.435654,0.609664,0.478720,0.659239,0.293652,0.615364,0.373597,0.390673,0.503613
LOG6,0.687626,0.326897,0.535316,0.471381,0.631890,0.460862,0.511020,0.578886,0.279687,0.278702,0.669645
LOG7,0.379981,0.741995,0.442507,0.633883,0.364120,0.431352,0.618681,0.527527,0.561500,0.519419,0.477514
LOG8,0.835584,0.253280,0.417528,0.515995,0.573820,0.350156,0.307997,0.643849,0.302016,0.245970,0.509690
TLOG1,0.614301,0.615870,0.537625,0.604327,0.347685,0.709481,0.404387,0.565687,0.667127,0.701727,0.303578


In [9]:
rice_cococonet = CococoNet_reader.read_cococonet('rice')

In [22]:
rice_cococonet.loc[['Os04g0518800','Os02g0628000','Os03g0109300','Os10g0479500','Os01g0588900','Os05g0591600','Os03g0857900','Os09g0547500','Os05g0541200','Os01g0708500','Os03g0697200']]

,Os01g0100100,Os01g0100200,Os01g0100300,Os01g0100400,Os01g0100466,Os01g0100500,Os01g0100600,Os01g0100650,Os01g0100700,Os01g0100800,...,CAA33921,CAA33922,EPlORYSAT000373786,CAA33923,CAA33928,CAA33925,ENSRNA050013646,CAA33926,EPlORYSAT000373848,CAA33927
Os04g0518800,0.376785,0.457624,0.471988,0.908532,0.486238,0.718271,0.161405,0.273118,0.895107,0.565454,...,0.349292,0.452513,0.497246,0.424711,0.546805,0.413383,0.614028,0.428666,0.614028,0.441178
Os02g0628000,0.731084,0.102935,0.496040,0.774908,0.445269,0.771787,0.353977,0.224151,0.646005,0.237783,...,0.420064,0.455733,0.473381,0.484581,0.434357,0.416556,0.614028,0.486468,0.614028,0.672509
Os03g0109300,0.830801,0.138896,0.491343,0.771054,0.429213,0.780602,0.816780,0.364743,0.406093,0.863457,...,0.509329,0.380616,0.482045,0.537350,0.649984,0.373619,0.614028,0.533515,0.614028,0.566464
Os10g0479500,0.905086,0.540971,0.500545,0.881829,0.446021,0.876061,0.427379,0.178609,0.940790,0.444977,...,0.374394,0.434006,0.482010,0.412918,0.331389,0.452305,0.614028,0.354277,0.614028,0.649399
Os01g0588900,0.676299,0.271887,0.475503,0.009580,0.459010,0.204434,0.718466,0.446003,0.013874,0.564487,...,0.649663,0.497375,0.470792,0.566126,0.526808,0.467445,0.614028,0.387523,0.614028,0.427735
Os05g0591600,0.014950,0.221395,0.500938,0.106513,0.454683,0.046070,0.050237,0.330507,0.005321,0.126075,...,0.343373,0.446580,0.472291,0.330567,0.492550,0.374791,0.614028,0.487955,0.614028,0.646608
Os03g0857900,0.685247,0.700198,0.505855,0.213620,0.483108,0.526624,0.893545,0.203073,0.343179,0.741333,...,0.368190,0.401914,0.471512,0.398243,0.563394,0.447832,0.614028,0.478492,0.614028,0.390047
Os09g0547500,0.210645,0.395262,0.534665,0.218068,0.442362,0.371895,0.684785,0.225077,0.093121,0.273263,...,0.500645,0.394414,0.501616,0.503847,0.513364,0.430358,0.614028,0.318916,0.614028,0.671154
Os05g0541200,0.870202,0.045000,0.530893,0.872394,0.465383,0.900712,0.813535,0.237931,0.645937,0.849254,...,0.553499,0.397240,0.471791,0.548819,0.584095,0.460760,0.614028,0.564417,0.614028,0.425682
Os01g0708500,0.766865,0.025912,0.534624,0.767664,0.478661,0.913096,0.935240,0.283762,0.883971,0.942330,...,0.368374,0.380545,0.470859,0.474937,0.527905,0.425588,0.614028,0.466694,0.614028,0.374421


In [24]:
rice_gene_list = ['Os04g0518800','Os02g0628000','Os03g0109300','Os10g0479500','Os01g0588900','Os05g0591600','Os03g0857900','Os09g0547500','Os05g0541200','Os01g0708500','Os03g0697200']

In [10]:
rice_cococonet

,Os01g0100100,Os01g0100200,Os01g0100300,Os01g0100400,Os01g0100466,Os01g0100500,Os01g0100600,Os01g0100650,Os01g0100700,Os01g0100800,...,CAA33921,CAA33922,EPlORYSAT000373786,CAA33923,CAA33928,CAA33925,ENSRNA050013646,CAA33926,EPlORYSAT000373848,CAA33927
Os01g0100100,1.000000,0.415044,0.480129,0.954701,0.448483,0.998227,0.995926,0.198254,0.940504,0.981818,...,0.382719,0.425544,0.470139,0.504815,0.409516,0.454757,0.614028,0.421163,0.614028,0.437491
Os01g0100200,0.415044,1.000000,0.475633,0.752125,0.390407,0.441919,0.234515,0.153863,0.540154,0.122671,...,0.356485,0.404344,0.474192,0.647188,0.664977,0.381620,0.614028,0.346238,0.614028,0.476471
Os01g0100300,0.480129,0.475633,1.000000,0.472164,0.491006,0.478127,0.477430,0.492454,0.475249,0.478270,...,0.482379,0.493690,0.494493,0.476882,0.506376,0.494493,0.614028,0.483356,0.614028,0.473212
Os01g0100400,0.954701,0.752125,0.472164,1.000000,0.467006,0.994425,0.922651,0.235191,0.979149,0.973359,...,0.401590,0.417485,0.472492,0.574758,0.499648,0.453827,0.614028,0.567841,0.614028,0.303638
Os01g0100466,0.448483,0.390407,0.491006,0.467006,1.000000,0.499860,0.479244,0.463978,0.468096,0.457304,...,0.561327,0.381993,0.485635,0.524495,0.430997,0.395004,0.614028,0.459460,0.614028,0.540134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAA33925,0.454757,0.381620,0.494493,0.453827,0.395004,0.466965,0.420893,0.401437,0.491166,0.443996,...,0.392821,0.405365,0.491993,0.433486,0.542915,1.000000,0.614028,0.406444,0.614028,0.410354
ENSRNA050013646,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,...,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,1.000000,0.614028,0.614028,0.614028
CAA33926,0.421163,0.346238,0.483356,0.567841,0.459460,0.545018,0.406758,0.329394,0.374544,0.497855,...,0.308608,0.453265,0.527565,0.408765,0.558342,0.406444,0.614028,1.000000,0.614028,0.519980
EPlORYSAT000373848,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,...,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,0.614028,1.000000,0.614028


In [26]:
tomato_genes_list = ['LOG8','LOC101262778','LOC101246083','LOG6','LOG7','TLOG1','LOC101252798']

In [27]:
test_list = [rice_gene_list, tomato_genes_list]
combinations = [p for p in itertools.product(*test_list)]

In [28]:
combo_df = pd.DataFrame(data = combinations, columns = ['arabi','tomato'])


In [29]:
combo_df.to_csv("/data/passala/git/Solanum_SV_Pan_Genome/Jack_prickles/rice_to_tomato_csv.csv",index = False)

In [ ]:
all_combo = itertools.permutations(tomato_genes_list,arabi_gene_list)

In [ ]:
tomato_cococonet_full.loc[['LOG8','LOC101262778','LOC101246083','LOG6','LOG7','TLOG1','LOC101252798']]

In [ ]:
matrix_of_locations = pd.read_csv('/data/passala/git/Coexpressalog_Method_Development/Cross_all_species/map_of_tomato_locations.csv', index_col= 0)#List of cross species maps
rice_tomato_table = Calculate_Score_list_for_thresholding('rice')
